# Transfer-learning using CNN

In this notebook, a transfer learning (TL) approach is followed to solve a cancer prediction task on gene-expression samples from a concrete tumor type. We perform a TL approach by pre-training a CNN on the non-Lung cancer samples from the TCGA PanCancer dataset, and then fine-tune the model on the Lung cancer dataset (see `PanCancer_Lung_Split` notebook). As input data, we use the gene expression profiles modeled as an image (matrix) per sample, whose pixels (gene expression values) are neighbours depending on their mean expression value across all the PanCancer samples (see `Mean_Treemap` R notebook).

In [1]:
import numpy as np
import pandas as pd
import warnings

# Auxiliary functions
from bio_dl_utils import *

Using TensorFlow backend.


## Progression free-interval

Here, we predict the discrete progression-free interval (PFI) of each patient (sample), which correponds to a binary classification task:

In [2]:
# Define survival variable of interest
surv_variable = "PFI"
surv_variable_time = "PFI.time"

### non-LUNG

We only use the non-LUNG tumor samples from the TCGA PanCancer dataset with the survival information of interest associated:

In [3]:
# Load samples-info dataset
Y_info = pd.read_hdf("../data/PanCancer/non_Lung_pancan.h5", 
                     key="sample_type")
Y_info.shape

(9374, 4)

In [4]:
# Load survival clinical outcome dataset
Y_surv = pd.read_hdf("../data/PanCancer/non_Lung_pancan.h5", 
                     key="sample_clinical")
Y_surv.shape

(9374, 33)

In [5]:
# tumor-normal distribution
Y_info.tumor_normal.value_counts(normalize=False, dropna=False)

Tumor     8771
Normal     603
Name: tumor_normal, dtype: int64

In [6]:
# Filter tumor samples from survival clinical outcome dataset
Y_surv = Y_surv.loc[Y_info.tumor_normal=="Tumor"]
Y_surv.shape

(8771, 33)

In [7]:
# Drop rows where surv_variable or surv_variable_time is NA
Y_surv.dropna(subset=[surv_variable, surv_variable_time], inplace=True)
Y_surv.shape

(8563, 33)

In [8]:
# Event PFI samples time distribution
Y_surv.loc[Y_surv.PFI==1.0]['PFI.time'].describe()

count     2992.000000
mean       625.818516
std        817.163242
min          0.000000
25%        188.000000
50%        370.000000
75%        729.250000
max      10334.000000
Name: PFI.time, dtype: float64

In [9]:
# Censored PFI samples time distribution
Y_surv.loc[Y_surv.PFI==0.0]['PFI.time'].describe()

count     5571.000000
mean      1050.329564
std       1017.383207
min          0.000000
25%        388.000000
50%        741.000000
75%       1409.000000
max      11217.000000
Name: PFI.time, dtype: float64

We create a discrete time class variable using the fixed-time point selected in `Lung_PFI_Prediction` notebook:

In [10]:
time = 230
Y_surv_disc = Y_surv[['PFI', 'PFI.time']].apply(
    lambda row: survival_fixed_time(time, row['PFI.time'], row['PFI']), axis=1)

Y_surv_disc.dropna(inplace=True)
Y_surv_disc.shape

(7707,)

In [11]:
# Event class fraction
sum(Y_surv_disc)/len(Y_surv_disc)

0.12222654729466718

In [12]:
%%time
# Load gene-exp matrices
n_width = 175
n_height = 175
dir_name = "mean_gene_exp_treemap_" + str(n_width) + "_" + str(n_height) + "_npy/"

X_gene_exp = np.array([np.load(dir_name + s.replace("-", ".") + ".npy") for s in Y_surv_disc.index]) 

CPU times: user 1.21 s, sys: 1.05 s, total: 2.26 s
Wall time: 2.26 s


In [13]:
X_gene_exp.shape

(7707, 175, 175)

We now create the binary class variables:

In [14]:
from sklearn.preprocessing import LabelEncoder

# Convert discrete time survival numerical variables into binary variables
Y_surv_disc_class = LabelEncoder().fit_transform(Y_surv_disc)
np.unique(Y_surv_disc_class)

array([0, 1])

In [15]:
Y_surv_disc_class.shape

(7707,)

### Lung

We also load the Lung tumor samples from the TCGA PanCancer dataset with the survival information of interest associated:

In [16]:
# Load samples-info dataset
Y_info_ft = pd.read_hdf("../data/PanCancer/Lung_pancan.h5", key="sample")
# Load survival clinical outcome dataset
Y_surv_ft = pd.read_hdf("../data/PanCancer/Lung_pancan.h5", key="survival_outcome")
# Filter tumor samples from survival clinical outcome dataset
Y_surv_ft = Y_surv_ft.loc[Y_info_ft.tumor_normal=="Tumor"]
# Drop rows where surv_variable or surv_variable_time is NA
Y_surv_ft.dropna(subset=[surv_variable, surv_variable_time], inplace=True)
Y_surv_ft.shape

(999, 33)

In [17]:
time = 230
Y_surv_disc_ft = Y_surv_ft[['PFI', 'PFI.time']].apply(
    lambda row: survival_fixed_time(time, row['PFI.time'], row['PFI']), axis=1)

Y_surv_disc_ft.dropna(inplace=True)
Y_surv_disc_ft.shape

(855,)

In [18]:
%%time
# Load gene-exp matrices
X_gene_exp_ft = np.array([np.load(dir_name + s.replace("-", ".") + ".npy") for s in Y_surv_disc_ft.index]) 

CPU times: user 166 ms, sys: 89.1 ms, total: 255 ms
Wall time: 255 ms


In [19]:
X_gene_exp_ft.shape

(855, 175, 175)

We now create the binary class variables:

In [20]:
# Convert discrete time survival numerical variables into binary variables
Y_surv_disc_class_ft = LabelEncoder().fit_transform(Y_surv_disc_ft)
Y_surv_disc_class_ft.shape

(855,)

### Join PT and FT

We use bayesian-optimization to perform the hyper-parameters tuning of a 2D-CNN architecture, using a cross-validation (CV) procedure. Random over-sampling is used both on pre-training and fine-tuning phases to deal with the severe class imbalance present in both non-Lung and Lung cancer datasets.

In [21]:
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import make_scorer
from imblearn.over_sampling import RandomOverSampler

# Define training datasets
X = X_gene_exp_ft
y = Y_surv_disc_class_ft

# Define the scaler
sc = StandardScaler()

# Define re-sampling method
ros = RandomOverSampler(random_state=69)

# Define image input shape
image_shape = (*X.shape[1:3], 1)

# Define custom transformer
ft = FunctionTransformer(func=reshape_transformer, kw_args={'final_shape': image_shape})

# Define cross-validation train-test splits
cv_split = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=23)

# Define evaluation metrics
model_sel_metric = 'auc'
eval_metric = {'auc': 'roc_auc', 
               'acc': make_scorer(opt_accuracy_score, needs_proba=True), 
               'sens': make_scorer(opt_sensitivity_score, needs_proba=True),
               'spec': make_scorer(opt_specificity_score, needs_proba=True),
               'prec': make_scorer(opt_precision_score, needs_proba=True),
               'f1': make_scorer(opt_f1_score, needs_proba=True),
               'mcc': make_scorer(opt_mcc_score, needs_proba=True),
               'thres': make_scorer(opt_threshold_score, needs_proba=True)}

# Bayesian-Optimization parameters
n_iter = 100
random_state = 666

## Over-sampling

### Fine-tune all

In [22]:
from hyperopt import hp

# Define the 2D-CNN hyperparameters space
params_space = {
    # PT hyper-param
    # We assume that the base model contains 2 CNN layers and 1 dense layer
    'clf__pt_params': hp.choice('pt_params', 
                [{'clf__dense_choice': hp.choice('dense_num_layers', 
                                         [{'clf__add_dense': 0,
                                           'clf__dense_unit_1': hp.choice('1dense_unit_1', [120, 160, 200, 240]),

                                           'clf__dense_dropout_1': hp.choice('1dense_dropout_1', [0.2, 0.4, 0.6, 0.8])
                                          },
                                          {'clf__add_dense': 1,
                                           'clf__dense_unit_1': hp.choice('2dense_unit_1', [120, 160, 200, 240]),
                                           'clf__dense_unit_2': hp.choice('2dense_unit_2', [25, 50, 75, 100]),

                                           'clf__dense_activation_2': 'relu',

                                           'clf__dense_dropout_1': hp.choice('2dense_dropout_1', [0.2, 0.4, 0.6, 0.8]),
                                           'clf__dense_dropout_2': hp.choice('2dense_dropout_2', [0.2, 0.4, 0.6, 0.8])
                                          }]),
                'clf__cnn_filter_1': hp.choice('cnn_filter_1', [2, 4, 8, 12, 16]),
                'clf__cnn_filter_2': hp.choice('cnn_filter_2', [8, 12, 16, 32, 40]),
                              
                # Input dim = 175, Output dim = [78, 86]
                'clf__cnn_kernel_1': hp.choice('cnn_kernel_1', [4, 8, 12, 16, 20]),
                # Input dim = [78, 86] (~ 175/2), Output dim = [32, 43]
                'clf__cnn_kernel_2': hp.choice('cnn_kernel_2', [2, 4, 8, 12, 16]),
                                   
                'clf__cnn_dropout_1': hp.choice('cnn_dropout_1', [0.2, 0.4, 0.6, 0.8]),
                'clf__cnn_dropout_2': hp.choice('cnn_dropout_2', [0.2, 0.4, 0.6, 0.8]),
                  
               'clf__batch_size': hp.choice('batch_size_pt', [64, 128, 256, 384, 512]),
               'clf__lr': hp.loguniform('lr_pt', np.log(1e-3), np.log(1e-1)),
               # Re-sampling hyper-params
               're_sample_pt__sampling_strategy': hp.choice('sampling_strategy_pt', [1, 1/2, 1/3, 1/4])}]),
    'clf__ft_params': hp.choice('ft_params', 
                [{'clf__batch_size': hp.choice('batch_size_ft', [32, 80, 128, 192, 256]),
                  'clf__lr': hp.loguniform('lr_ft', np.log(5e-4), np.log(1e-1)),
                  # Re-sampling hyper-params
                  're_sample_ft__sampling_strategy': hp.choice('sampling_strategy_ft', [1, 1/2, 1/3, 1/4])}])
}

In [23]:
from imblearn.pipeline import Pipeline

warnings.filterwarnings('ignore')

# Define PT 2D-CNN estimator
pre_model_path = 'keras-models/mean_ros_new_cnn_pt_ft_disc_pfi_x.h5'
cnn_pt = SklearnCNN(input_shape=image_shape, cnn_filter={}, cnn_kernel={}, cnn_pool={1: 2, 2: 2}, 
                 cnn_activation={1: 'relu', 2: 'relu'}, cnn_dropout={}, dense_unit={}, dense_activation={1: 'relu'}, 
                 dense_dropout={}, output_unit=1, output_activation='sigmoid', 
                 optimizer_name='adam', loss_function='binary_crossentropy', epoch=200, patience=10, verbose=0, 
                 model_path=pre_model_path)

cnn_pipe_pt = Pipeline([('scaler', sc), ('re_sample_pt', ros), ('transf', ft), ('clf', cnn_pt)])

# Define FT 2D-CNN estimator
cnn_ft = SklearnFT(pre_layer=17, n_freeze=0, dense_unit={}, dense_activation={}, dense_dropout={},
                      output_unit=1, output_activation='sigmoid', optimizer_name='adam', 
                      loss_function='binary_crossentropy', epoch=200, patience=10, verbose=0,
                      pre_model=pre_model_path, 
                      model_path='keras-models/mean_ros_new_cnn_pt_ft_disc_pfi_x_fine.h5')

cnn_pipe_ft = Pipeline([('scaler', sc), ('re_sample_ft', ros), ('transf', ft), ('clf', cnn_ft)])

# Define Bayesian-Optimization
hyper_search = HyperoptCV_TL(estimator_pt=cnn_pipe_pt, 
                          X_pt=X_gene_exp.reshape(X_gene_exp.shape[0], X_gene_exp.shape[1]*X_gene_exp.shape[2]), 
                          y_pt=Y_surv_disc_class, estimator_ft=cnn_pipe_ft, 
                          hyper_space=params_space, cv=cv_split, scoring=eval_metric, opt_metric=model_sel_metric,
                          n_iter=n_iter, random_seed=random_state, verbose_file = 'mean_class_imb_pt_new_pt_ft_verbose.txt')

In [ ]:
%%time
best_trial = hyper_search.model_selection(X.reshape(X.shape[0], X.shape[1]*X.shape[2]), y)

In [25]:
best_trial['result']['params']

{'clf__ft_params': {'clf__batch_size': 192,
  'clf__lr': 0.0015052167703729666,
  're_sample_ft__sampling_strategy': 1},
 'clf__pt_params': {'clf__batch_size': 384,
  'clf__cnn_dropout_1': 0.2,
  'clf__cnn_dropout_2': 0.2,
  'clf__cnn_filter_1': 4,
  'clf__cnn_filter_2': 16,
  'clf__cnn_kernel_1': 4,
  'clf__cnn_kernel_2': 4,
  'clf__dense_choice': {'clf__add_dense': 1,
   'clf__dense_activation_2': 'relu',
   'clf__dense_dropout_1': 0.8,
   'clf__dense_dropout_2': 0.4,
   'clf__dense_unit_1': 120,
   'clf__dense_unit_2': 25},
  'clf__lr': 0.00212194465594212,
  're_sample_pt__sampling_strategy': 0.25}}

In [27]:
scores = best_trial['result']['test_score']
res = pd.DataFrame({'AUC': scores['test_auc'], 
              'ACC': scores['test_acc'], 
              'Sens': scores['test_sens'], 
              'Spec': scores['test_spec'],
              'Prec': scores['test_prec'],
              'F-1': scores['test_f1'],
              'MCC': scores['test_mcc'],
              'Thres': scores['test_thres']})
res.describe()

,ACC,AUC,F-1,MCC,Prec,Sens,Spec,Thres
count,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000
mean,0.715789,0.699777,0.293932,0.223031,0.206692,0.603750,0.727520,0.053323
std,0.095348,0.058382,0.042768,0.054835,0.059189,0.154931,0.118836,0.067804
min,0.485380,0.572984,0.200000,0.088983,0.117021,0.312500,0.451613,0.001499
25%,0.656433,0.651310,0.265646,0.191117,0.172414,0.500000,0.651613,0.011440
50%,0.716374,0.705880,0.290616,0.227136,0.186773,0.625000,0.729032,0.024575
75%,0.771930,0.740784,0.326850,0.258169,0.226648,0.687500,0.798387,0.056901
max,0.888889,0.839919,0.384615,0.326640,0.384615,0.937500,0.948387,0.317080


In [28]:
# Save results
file_path = 'results/ros_mean_new_disc_pfi_lung_ft_100_iter_rep_kfold.csv'
res.to_csv(file_path, index=False)